In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm.notebook import tqdm

In [2]:
path_train = r"../train/"

In [3]:
dislok = pd.read_parquet(path_train + '/dislok_wagons.parquet').convert_dtypes()
wag_prob = pd.read_parquet(path_train + '/wagons_probeg_ownersip.parquet').convert_dtypes()
wag_param = pd.read_parquet(path_train + '/wag_params.parquet').convert_dtypes()
freight_data = pd.read_parquet(path_train + '/freight_info.parquet').convert_dtypes()

dislok.fillna(pd.NA, inplace=True)

In [4]:
dislok.shape

(8205541, 16)

In [ ]:
dislok = dislok.merge(freight_data, on='fr_id')

In [ ]:
dislok.shape

In [ ]:
target_df = pd.read_csv(path_train +'/target/y_train.csv').convert_dtypes()
target_df['month'] = pd.to_datetime(target_df['month'])
target_df.sample(5)

In [ ]:
target_df.target_month.mean()

### Дислокация вагонов - сбор данных

In [ ]:
dislok.columns

In [ ]:
import gc
gc.collect()

In [ ]:
dislok['days_to_pl_rem'] = (dislok['date_pl_rem'] - dislok['plan_date']).dt.days
dislok['another_road'] = dislok['id_road_dest'] != dislok['id_road_send']
dislok['last_rem_date'] = dislok[['date_kap', 'date_dep']].max(axis=1)
dislok['days_from_last_rem'] = (dislok['plan_date'] - dislok['last_rem_date']).dt.days
dislok = dislok.convert_dtypes()

In [ ]:
dislok.sort_values(by='plan_date', inplace=True)

In [ ]:
to_fill = ['ost_prob']

dislok[to_fill] = dislok.groupby('wagnum')[to_fill].transform(lambda x: x.fillna(method='ffill'))
dislok[to_fill] = dislok.groupby('wagnum')[to_fill].transform(lambda x: x.fillna(0))

In [ ]:
dislok.info()

In [ ]:
wagnum_subtables = {wagnum: table for wagnum, table in tqdm(dislok.groupby('wagnum'))}

In [ ]:
from itertools import product

intervals = [pd.DateOffset(days=10), pd.DateOffset(days=30), pd.DateOffset(days=90)]
functions = {"isload": ['mean'], "ost_prob": ['min', 'max'], 'freight': ['mean']}

fr_classes = ['skoroport', 'naval', 'nasip', 'naliv', 'openvagons', 'soprovod', 'smerz']
for fr_class in fr_classes:
    functions[fr_class] = ['mean']

take_last_cols = ['date_kap', 'date_dep', 'kod_vrab', 'date_pl_rem', 'days_to_pl_rem', 'last_rem_date', 'days_from_last_rem', 'ost_prob']
rolling_cols = set()


for wagnum, table in tqdm(wagnum_subtables.items()):
    for interval in intervals:
        for col, func_list in functions.items():
            for func in func_list:
                table[f"{interval.days}d_{col}_{func}"] = getattr(table[col].rolling(interval.days), func)(engine='numba')
                rolling_cols.add(f"{interval.days}d_{col}_{func}")

for wagnum, table in tqdm(wagnum_subtables.items()):
    table['10d_ost_prob_sum'] = table['10d_ost_prob_max'] - table['10d_ost_prob_min']
    table['30d_ost_prob_sum'] = table['30d_ost_prob_max'] - table['30d_ost_prob_min']
    table['90d_ost_prob_sum'] = table['90d_ost_prob_max'] - table['90d_ost_prob_min']
    table['10d_prob_mean'] = table['10d_ost_prob_sum'] / 10
    table['30d_prob_mean'] = table['30d_ost_prob_sum'] / 30
    table['90d_prob_mean'] = table['90d_ost_prob_sum'] / 90

for newcol in ['10d_prob_mean', '30d_prob_mean', '90d_prob_mean', '10d_ost_prob_sum', '30d_ost_prob_sum', '90d_ost_prob_sum']:
    rolling_cols.add(newcol)

columns_to_add = list(rolling_cols) + take_last_cols

In [ ]:
wagnum_subtables[0][['plan_date', '30d_ost_prob_min', '30d_ost_prob_max', '30d_prob_mean', '30d_ost_prob_sum']].sample(10)

In [ ]:
columns_to_add

In [ ]:
target_df.sort_values(by=['month', 'wagnum'], inplace=True)

data = pd.concat([wagnum_subtables[wagnum] for wagnum in tqdm(wagnum_subtables.keys())])
data = data[columns_to_add + ['wagnum', 'plan_date']]
data.rename({'plan_date': 'month'}, axis=1, inplace=True)
target_df_new = target_df.merge(data, on=['month', 'wagnum'], how='left')
    

target_df_new.sample(5)

In [ ]:
target_df_new.info()

In [ ]:
target_df_new.month.unique()

### Тип владения вагоном - сбор данных

In [ ]:
wag_prob_months.dtypes

In [ ]:
target_df_new.dtypes

In [ ]:
wag_prob_months = wag_prob[wag_prob.repdate.dt.day == 1].copy()
print(wag_prob_months.shape)

wag_prob_months.drop(['ost_prob', 'month'], axis=1, inplace=True)
wag_prob_months.rename({'repdate': 'month'}, axis=1, inplace=True)

target_df_new_ = target_df_new.merge(wag_prob_months, on=['month', 'wagnum'], how='left')
target_df_new_.sample(5)

In [ ]:
target_df_new_.info()

In [ ]:
target_df_new = target_df_new_

### Характеристики вагонов - обзор данных

In [ ]:
wag_param.columns

In [ ]:
target_df_new = target_df_new.merge(wag_param, on=['wagnum'], how='left')
target_df_new.sample(5)

In [ ]:
target_df_new.info()

In [ ]:
target_df_new['expected_srok_sl_y'] = (target_df_new['srok_sl'] - target_df_new['date_build']).dt.days / 365
target_df_new['expected_srok_sl_y'].plot.hist(bins=50, range=(18, 36))

In [ ]:
target_df_new['expected_srok_sl_y_b'] = target_df_new['expected_srok_sl_y'].apply(lambda x: "short" if x <= 27.5 else "long")
target_df_new['expected_srok_sl_y_b'].value_counts()

In [ ]:
target_df_new['date_build_b'] = pd.cut(target_df_new['date_build'].dt.year, bins=[0, 2000, 2007, 2017, 2030])
target_df_new['date_build_b'].value_counts()

In [ ]:
target_df_new.to_csv(path_train + '/data/first_3_tables.csv', index=False)